Lecture Notes:

- Num of sample partition: K

1. Cross-validation
- Bias-Variance trade off
- for prediction error estimation
- CV for regression
    - Estimate CV(K), which is the average of all MSE_k
- CV for classification
    - Estimate CV(K), which is the average of all Err_k (error rate)

2. Bootstrap
- Resampling with replacements
- for SE estimation, not good for prediction erorr estimation
- bootstrap confidence interval
    - resampling 1000 times from originial data
    - build model for each sample and compute RSE/MSE
    - compute the mean and std of the set of 1000 RSE/MSE
    - or use 95% threshold to get lower and upper bound of the set of 1000 RSE/MSE, i.e. 2.5-97.5 percentile of the set

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets,preprocessing
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.utils import resample
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices 

In [38]:
data = pd.read_csv('data/Auto.csv',header=0,na_values='?')
data = data.dropna().reset_index(drop=True)
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [32]:
# Cross-validataion
X = pd.DataFrame(data.horsepower)
Y = data.mpg

model = LinearRegression()
# LOO
test1 = cross_val_score(model,X,Y,cv=X.shape[0],scoring='neg_mean_squared_error',n_jobs=-1)
# k-fold
k_fold = 5
test2 = cross_val_score(model,X,Y,cv=k_fold,scoring='neg_mean_squared_error',n_jobs=-1)

print(-test1.mean(), -test2.mean())

# Note: above is assuming no data preprocessing for each fold. Otherwise, use pipeline
# and preprocess each fold
model_pipe = make_pipeline(preprocessing.StandardScaler(),LinearRegression())
test_pipe = cross_val_score(model,X,Y,cv=k_fold,scoring='neg_mean_squared_error',n_jobs=-1)
print(-test_pipe.mean())

24.2315135179 31.4470140886
31.4470140886


In [45]:
# Bootstrap
n_iter = 100
stats = list()
for i in range(n_iter):
    data_resample = resample(data)
    X_resample = pd.DataFrame(data_resample.horsepower)
    Y_resample = data_resample.mpg
    modelBoot = LinearRegression().fit(X_resample,Y_resample)
    stats.append(modelBoot.score(X_resample,Y_resample))
print(np.mean(stats),np.std(stats))


0.604593763029 0.0238972578865
